In [2]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/GRU

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[]

In [3]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [4]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [5]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).ravel()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = data_x.to_numpy()
    data_x = convert_3D(normalize(data_x))
    return data_x, labels

In [ ]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
testing_data_x, testing_data_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")
validation_data_x, validation_data_y = features_and_labels("../Kepler-Val.pkl", "../Labels-Val.npy")

In [6]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
276/276 [==============================] - 10s 21ms/step - loss: 0.6840 - accuracy: 0.7013 - val_loss: 0.6736 - val_accuracy: 0.7715
Epoch 2/10
276/276 [==============================] - 4s 16ms/step - loss: 0.6618 - accuracy: 0.7704 - val_loss: 0.6484 - val_accuracy: 0.7715
Epoch 3/10
276/276 [==============================] - 4s 16ms/step - loss: 0.6313 - accuracy: 0.7704 - val_loss: 0.6118 - val_accuracy: 0.7715
Epoch 4/10
276/276 [==============================] - 4s 16ms/step - loss: 0.5886 - accuracy: 0.7704 - val_loss: 0.5642 - val_accuracy: 0.7715
Epoch 5/10
276/276 [==============================] - 4s 16ms/step - loss: 0.5441 - accuracy: 0.7704 - val_loss: 0.5280 - val_accuracy: 0.7715
Epoch 6/10
276/276 [==============================] - 4s 16ms/step - loss: 0.5207 - accuracy: 0.7704 - val_loss: 0.5148 - val_accuracy: 0.7715
Epoch 7/10
276/276 [==============================] - 4s 16ms/step - loss: 0.5114 - accuracy: 0.7704 - val_loss: 0.5062 - val_accuracy: 0.771

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [7]:
%tensorboard --logdir logs/GRU/fit